# Painel Fotovoltaico
### Autora: Luana Dias Costa Lucas <br> Empresa : WP SOLUÇÕES INDUSTRIAIS INTELIGENTES LTDA


## RELATÓRIO MANUTENÇÃO

Gera um pdf com todos os paineis que precisam de manutenção baseado na data da ultima manutenção e na eficiencia dos inversores

In [1]:
#Bibliotecas
from ipynb.fs.full.Helper import *
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from fpdf import FPDF
import pandas as pd
import os
import numpy as np

#maintenance = pd.read_csv('/Plant_1_Generation_Data_Maintenance.csv')
#generationData = pd.read_csv('/Plant_1_Generation_Data.csv')
#plant = '1'

In [8]:
# Escolha da central solar
plant = whichPlant()        
if plant == '1':
    path = 'archive/Plant_1/'
else:
    path = 'archive/Plant_2/'
maintenance = pd.read_csv(path + 'Generation_Data_Maintenance.csv')
generationData = pd.read_csv(path + 'Generation_Data.csv')


    Por favor digite 1 ou 2 para escolher uma das 2 centrais solares
    


In [9]:
# Add uma nova coluna com a eficiencia da geracao de energia
generationData_sort = generationData.loc[generationData['AC_POWER'] > float(0)]
generationData_sort = generationData_sort.sort_values(by=['SOURCE_KEY']).reset_index().drop(columns = ['index', 'PLANT_ID'])

x = len(generationData_sort)
eff=[]
for i in range(x):
    eff.append((generationData_sort.iloc[i]['AC_POWER'])/(generationData_sort.iloc[i]['DC_POWER']))

generationData_sort['Efficiency'] = eff

In [10]:
# Ve se existe a necessidade de marcar uma manutencao preventiva
maintenance_prevent = maintenance
x = len(maintenance.index)

for i in range(x):
    lastMaintenance = datetime.strptime(maintenance_prevent['LAST_MAINTENANCE'][i], '%Y-%m-%d').date()
    gap = maintenance_prevent['GAP_MONTHS'][i]
    if lastMaintenance + relativedelta(months=+gap) <= date.today():
      maintenance_prevent.loc[i, 'MAINTENANCE_REQUIRED'] = 'Sim'
      maintenance_prevent.loc[i, 'MOTIVE'] = 'Preventiva'
    else:
      maintenance_prevent.loc[i, 'MAINTENANCE_REQUIRED'] = 'Não'
      maintenance_prevent.loc[i, 'MOTIVE'] = np.nan

In [11]:
# Confere se precisa de uma manutenção baseado na eficiencia (considerando que uma eficiencia abaixo de 60% é indesejada)
maintenance_updated = maintenance_prevent
maintenance_needed = generationData_sort.loc[generationData_sort['Efficiency'] < float(0.6)]
Source = maintenance_needed['SOURCE_KEY'].unique() 
for x in Source:
    maintenance_updated.loc[maintenance_updated['SOURCE_KEY'] == x,'MAINTENANCE_REQUIRED'] = 'Sim'
    df = maintenance_updated.loc[maintenance_updated['SOURCE_KEY'] == x,'MOTIVE']
    if df.isnull().values:
        maintenance_updated.loc[maintenance_updated['SOURCE_KEY'] == x,'MOTIVE'] = 'Baixa Eficiência'
    else:
        maintenance_updated.loc[maintenance_updated['SOURCE_KEY'] == x,'MOTIVE'] = df.values + ' e Baixa Eficiência'
    
maintenance_updated.to_csv(path + 'Generation_Data_Maintenance.csv', index=False)

In [12]:
maintenance_required = maintenance_updated.drop(maintenance_updated[maintenance_updated['MAINTENANCE_REQUIRED'] != 'Sim'].index)
maintenance_required = maintenance_required.drop(columns = ['MAINTENANCE_REQUIRED', 'GAP_MONTHS']).reset_index()

In [13]:
pdf = FPDF()
# Set left margin
pdf.set_left_margin(0)
pdf.add_page()
pdf.set_font('Arial', 'B', 14)

## Title
pdf.cell(0,15,'Conjunto de paineis que precisam de manutenção - ' + date.today().strftime("%d/%m/%Y"), align='C')
pdf.ln(20)
pdf.set_font('Arial', '', 10)
pdf.cell(10)

main_req_pdf = maintenance_required.drop(columns = ['index']).reset_index()
x = len(main_req_pdf.index)

pdf.cell(0,0,'São ao total ' + str(x) + ' painéis que precisam de manutenção, abaixo está a lista com a Source Key e a data da última manutenção' , align='L')
pdf.ln(10)

spreadsheet_df_to_pdf(pdf, main_req_pdf, 3, 30, True)

dataHoje = date.today().strftime("%d%m%Y")

if not os.path.exists('Relatorios/Manutencao/Plant' + plant):
   os.makedirs('Relatorios/Manutencao/Plant' + plant)
pdf.output('Relatorios/Manutencao/Plant'+ plant +'/Conjunto de paineis que precisam de manutenção ' + dataHoje + '.pdf', 'F')

''